In [1]:
### This script download traffic volume ground truth from National Highway using API.
### Note that the sensor sequence in the h5 files does not match the sequence in sensors_498.csv, but the site ids are recorded in h5 files.

import pandas as pd
from tqdm import tqdm
import numpy as np
import requests
import tensorflow as tf

df = pd.read_csv('sensors_498.csv')
start = '2022-01-01 00:14:00'
end = '2022-12-31 23:59:00'

In [2]:
# Generate a DataFrame with a fixed set of timestamps for every 15 minutes of y, considering UK DST adjustments
def generate_fixed_timestamps(start, end):
    # Create range for the whole year with 15-minute intervals
    dt_range = pd.date_range(start=start, end=end, freq='15T')

    fixed_df = pd.DataFrame({'Timestamp': dt_range})
    return fixed_df

In [3]:
# Function to fetch data for a single site
def fetch_data_for_site(site_id):
    page = 1
    while True:
        url = f"{base_url}?sites={site_id}&page={page}&page_size=5000"
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Data download stopped at page {page-1} for site {site_id}")
            break
        
        data = response.json()
        if not data['Rows']:
            break
        
        result = pd.DataFrame(data['Rows'])
        result.replace({'': np.nan}, inplace=True)
        yield result
        page += 1


In [4]:
base_url = f'https://webtris.nationalhighways.co.uk/api/v1.0/reports/{start[8:10]+start[5:7]+start[0:4]}/to/{end[8:10]+end[5:7]+end[0:4]}/daily'

# Create the fixed timestamp DataFrame
fixed_timestamps_df = generate_fixed_timestamps(start, end)
t = len(fixed_timestamps_df)

# Loop through all sensor IDs and fetch data
sites_list = df.Id.tolist()  # Your list of sensor IDs
data_frames = []

for site_id in tqdm(sites_list):
    
    site_id = str(site_id)
    if site_id[0:3] == '999': # 999 is an indicator for non-valid sensor, which is replaced with the sensor that has the opposite direction
        site_id = site_id[3:]
        print('999'+site_id + ' changed to' + site_id + '.')
        
    site_data_frames = []
    for df in fetch_data_for_site(site_id):
        df = df[['Site Name', 'Report Date', 'Time Period Ending', 'Avg mph', 'Total Volume']]
        site_data_frames.append(df)
        
    if site_data_frames:  # Only append if we have data
        # Concatenate all pages for the site to one df
        df = pd.concat(site_data_frames, ignore_index=True)
        
        # Combine 'Report Date' and 'Time Period Ending' into a full datetime
        df['Timestamp'] = pd.to_datetime(df['Report Date'].str.slice(0, 10) + ' ' + df['Time Period Ending'].str.slice(0, 5))

        # Identify duplicates based on 'Timestamp'
        duplicate_rows = df.duplicated(subset='Timestamp', keep=False)
        duplicate_df = df[duplicate_rows]
        
        # For duplicates, average non-NaN values
        if not duplicate_df.empty:
            averaged_df = duplicate_df.groupby('Timestamp').agg({
                'Avg mph': lambda x: np.nanmean(x.astype(float)),
                'Total Volume': lambda x: np.nanmean(x.astype(float)),
                'Site Name': 'first',  # Keep the first site name
                'Report Date': 'first',  # Keep the first report date
                'Time Period Ending': 'first'  # Keep the first time period ending
            }).reset_index()
            
            # Remove original duplicates from df
            df = df[~duplicate_rows]
            
            # Append averaged duplicates back to df
            df = pd.concat([df, averaged_df], ignore_index=True)
            
        # Ensure the dataframe is sorted by Timestamp after handling duplicates
        df.sort_values(by='Timestamp', inplace=True)
        
        # Merge with the fixed timestamps DataFrame
        merged_df = pd.merge(fixed_timestamps_df, df, on='Timestamp', how='left')
        if merged_df['Avg mph'].isna().all() and df['Total Volume'].isna().all():
            print(site_id + ' has no valid data.')
        merged_df.drop(columns=['Report Date','Time Period Ending','Timestamp','Site Name'], inplace=True)

        data_frames.append(merged_df)
        
    else:
        df_nan = pd.DataFrame({
            'Avg mph': np.nan,
            'Total Volume': np.nan
        }, index=range(t))
        data_frames.append(df_nan)
        print(site_id + ' is not valid.')

  0%|          | 1/498 [00:19<2:40:41, 19.40s/it]

Data download stopped at page 8 for site 8825


  0%|          | 2/498 [00:38<2:39:08, 19.25s/it]

Data download stopped at page 8 for site 8826


  1%|          | 3/498 [00:57<2:38:07, 19.17s/it]

...

Data download stopped at page 7 for site 10726


100%|█████████▉| 497/498 [2:35:18<00:18, 18.23s/it]

Data download stopped at page 7 for site 18388


100%|██████████| 498/498 [2:35:37<00:00, 18.75s/it]

Data download stopped at page 7 for site 18526


In [5]:
# Convert the cleaned and validated DataFrame into tensors
tensors = [tf.convert_to_tensor(df[['Avg mph', 'Total Volume']].astype(float).values, dtype=tf.float32) for df in data_frames]
tensor = tf.stack(tensors)

# Convert to tensors if needed
timestamps_tensor = tf.constant(fixed_timestamps_df['Timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S').values)
sites_tensor = tf.constant(sites_list)


In [6]:
import h5py

with h5py.File(f'sensor498_{start[:10]}_{end[:10]}.h5', 'w') as f:  
    data = f.create_dataset('data', shape=tensor.shape)  
    data[:] = tensor.numpy()
    timestamps = f.create_dataset('timestamps', shape=timestamps_tensor.shape, dtype=h5py.special_dtype(vlen=str)) 
    timestamps[:] = timestamps_tensor
    sites = f.create_dataset('sites', shape=sites_tensor.shape) 
    sites[:] = sites_tensor

In [7]:
# Load the tensor from the HDF5 file  
with h5py.File(f'sensor498_{start[:10]}_{end[:10]}.h5', 'r') as f:  
    data = tf.convert_to_tensor(f['data'][:])  
    sites = tf.convert_to_tensor(f['sites'][:]) 
    timestamps = tf.convert_to_tensor(f['timestamps'][:]) 

In [8]:
print(data.shape)
print(sites.shape)
print(timestamps.shape)

(498, 35040, 2)
(498,)
(35040,)


In [9]:
nan_mask = tf.math.is_nan(data)  
num_nan = tf.reduce_sum(tf.cast(nan_mask, tf.int32))  
print('Missing data ratio: '+ str(num_nan.numpy()/tf.size(data).numpy()*100))

Missing data ratio: 14.71192131539858
